In [1]:
import os

from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt

import numpy as np
import torch
import torch.nn as nn

from models.pointnet import PointNet, TNet
from models.modules import SpecialEuclideanGeodesicLoss

from runners.experiment import run_epoch

from torchinfo import summary

def _R(theta, phi, psi, N):
    return (torch.tensor([
        [np.cos(theta), -np.sin(theta), 0],
        [np.sin(theta), np.cos(theta), 0],
        [0, 0, 1]
    ]) @ torch.tensor([
        [np.cos(phi), 0, np.sin(phi)],
        [0, 1, 0],
        [-np.sin(phi), 0, np.cos(phi)]
    ]) @ torch.tensor([
        [1, 0, 0],
        [0, np.cos(psi), -np.sin(psi)],
        [0, np.sin(psi), np.cos(psi)]
    ])).unsqueeze(0).float().repeat(N, 1, 1)

def _T(x, y, z, N):
    return torch.tensor([[x], [y], [z]]).unsqueeze(0).float().repeat(N, 1, 1)

def RT(angles, translations, N):
    r = _R(*angles, N)
    t = _T(*translations, N)
    return torch.cat((r, t), dim=-1)


In [9]:
x = torch.ones(1)

x = torch.stack([x, x, x, x])

x.mean()


tensor(1.)

In [5]:
SHAPE = 159, 3
B = 2

LR = 0.0001
WD = 0.0001

model = PointNet()
criterion = SpecialEuclideanGeodesicLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=WD)

x = torch.ones((B, *SHAPE))

angles = 0.1, 0.1, 0.1
translations = 0.1, 0.2, 0.3

y = RT(angles, translations, B)

# print(y[0])

b = RT((0, 0, 0), (0, 0, 0), 4)[:, :3, :3]


torch.diagonal(b, dim1=1, dim2=2).sum(1)

# model.eval()
print(model)
model(x).shape

# for _ in range(1000):
#     optimizer.zero_grad()
#     outputs = model(x)
#     loss = criterion(outputs, y)
#     loss.backward()
#     optimizer.step()

# model(x)


PointNet(
  (permute): Permute()
  (input_tnet): TNet(
    (permute): Permute()
    (conv): Sequential(
      (0): Conv1d(3, 64, kernel_size=(1,), stride=(1,))
      (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
      (3): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
      (4): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): LeakyReLU(negative_slope=0.01)
      (6): Conv1d(128, 1024, kernel_size=(1,), stride=(1,))
      (7): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (8): LeakyReLU(negative_slope=0.01)
    )
    (maxpool): LambdaModule()
    (fc): Sequential(
      (0): Linear(in_features=1024, out_features=512, bias=True)
      (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
      (3): Linear(in_features=512, out_features=256, bias=True)

torch.Size([2, 64, 159])

In [3]:
print(y)


tensor([[[ 0.9900, -0.0894,  0.1088,  0.1000],
         [ 0.0993,  0.9910, -0.0894,  0.2000],
         [-0.0998,  0.0993,  0.9900,  0.3000]],

        [[ 0.9900, -0.0894,  0.1088,  0.1000],
         [ 0.0993,  0.9910, -0.0894,  0.2000],
         [-0.0998,  0.0993,  0.9900,  0.3000]]])
